In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from dataclasses import dataclass
import scipy.stats as stats
import scipy.integrate as intg
import math
import cmath

def heston_euler_call(S0, v0, r, kappa, theta, sigma, rho, T, K, N=10000, m=100, return_paths=False):
    dt = T / m
    lnS = np.full((N, m+1), np.log(S0))
    v = np.full((N, m+1), v0)
    for t in range(1, m+1):
        Z1 = np.random.normal(size=N)
        Z2 = np.random.normal(size=N)
        dW1 = np.sqrt(dt) * Z1
        dW2 = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)
        v_prev = v[:, t-1]
        v_new = v_prev + kappa * (theta - v_prev) * dt + sigma * np.sqrt(np.abs(v_prev)) * dW2
        v[:, t] = np.maximum(v_new, 0)
        lnS[:, t] = lnS[:, t-1] + (r - 0.5 * v_prev) * dt + np.sqrt(np.abs(v_prev)) * dW1
    S_T = np.exp(lnS[:, -1])
    payoffs = np.maximum(S_T - K, 0)
    price = np.exp(-r * T) * np.mean(payoffs)
    if return_paths:
        S_paths = np.exp(lnS)
        return price, S_paths, v
    return price

def heston_qe_call(S0, v0, r, kappa, theta, sigma, rho, T, K, N=1000000, m=100, return_paths=False):
    dt = T / m
    lnS = np.full((N, m+1), np.log(S0))
    v = np.full((N, m+1), v0)
    for t in range(1, m+1):
        v_prev = v[:, t-1]
        m_v = theta + (v_prev - theta) * np.exp(-kappa * dt)
        s2_v = (v_prev * sigma ** 2 * np.exp(-kappa * dt) / kappa) * (1 - np.exp(-kappa * dt)) \
             + theta * sigma ** 2 * (1 - np.exp(-kappa * dt)) ** 2 / (2 * kappa)
        psi = s2_v / m_v ** 2
        Z = np.random.normal(size=N)
        U = np.random.uniform(size=N)
        v_new = np.zeros_like(v_prev)
        phi_c = 1.5
        mask = psi <= phi_c
        if np.any(mask):
            b2 = 2 / psi[mask] - 1 + np.sqrt(2 / psi[mask]) * np.sqrt(2 / psi[mask] - 1)
            a = m_v[mask] / (1 + b2)
            v_new[mask] = a * (np.sqrt(b2) + Z[mask]) ** 2
        if np.any(~mask):
            p = (psi[~mask] - 1) / (psi[~mask] + 1)
            beta = (1 - p) / m_v[~mask]
            u = U[~mask]
            v_new[~mask] = np.where(u <= p, 0, -np.log((1 - u) / (1 - p)) / beta)
        v[:, t] = v_new
        Z1 = np.random.normal(size=N)
        Z2 = np.random.normal(size=N)
        dW1 = np.sqrt(dt) * Z1
        dW2 = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho ** 2) * Z2)
        v_avg = (v_prev + v_new) / 2
        lnS[:, t] = lnS[:, t-1] + (r - 0.5 * v_avg) * dt + np.sqrt(np.abs(v_avg)) * dW1
    S_T = np.exp(lnS[:, -1])
    payoffs = np.maximum(S_T - K, 0)
    price = np.exp(-r * T) * np.mean(payoffs)
    if return_paths:
        S_paths = np.exp(lnS)
        return price, S_paths, v
    return price

@dataclass
class HestonParams:
    v: float
    kappa: float
    theta: float
    sigma: float
    rho: float


def heston_cf(s, v, kappa, theta, sigma, rho, u, t):
    d = cmath.sqrt((rho * sigma * u * 1j - kappa) ** 2 + sigma ** 2 * (u * 1j + u ** 2))
    g = ((rho * sigma * u * 1j - kappa + d) / (rho * sigma * u * 1j - kappa - d))
    C = (kappa * theta / sigma ** 2 * (
        t * (kappa - rho * sigma * u * 1j - d) -
        2 * cmath.log((1 - g * cmath.exp(-d * t)) / (1 - g))))
    D = ((kappa - rho * sigma * u * 1j - d) / sigma ** 2 *
         ((1 - cmath.exp(-d * t)) / (1 - g * cmath.exp(-d * t))))
    return cmath.exp(C + D * v + u * math.log(s) * 1j)


def heston_integrand(u, t, k, s, r, v, kappa, theta, sigma, rho):
    return (cmath.exp(-1j * u * math.log(k)) / (1j * u) *
            (cmath.exp(r * t) * heston_cf(s, v, kappa, theta, sigma, rho, u - 1j, t) -
             k * heston_cf(s, v, kappa, theta, sigma, rho, u, t))).real


def heston_scalar(t, k, s, r, is_call, heston_params):
    integral = intg.quad(
        heston_integrand, 0, 100,
        args=(t, k, s, r, heston_params.v, heston_params.kappa, heston_params.theta,
              heston_params.sigma, heston_params.rho), limit=200, epsabs=1e-8, epsrel=1e-8)[0]
    call_price = (0.5 * (s - math.exp(- r * t) * k) +
                  1 / math.pi * math.exp(- r * t) * integral)
    if is_call:
        return call_price
    return call_price + k * np.exp(- r * t) - s

params = {
    'S0': 1,
    'v0': 0.1,
    'r': 0.045,
    'kappa': 3.0,
    'theta': 0.3,
    'sigma': 0.5,
    'rho': -0.2,
    'T': 1.0,
    'K': 0.8
}


heston_params = HestonParams(
    v=params['v0'],
    kappa=params['kappa'],
    theta=params['theta'],
    sigma=params['sigma'],
    rho=params['rho'],
)

N_values = [1000, 5000, 10000, 20000]
m_values = [50, 100, 200, 500]
table = PrettyTable()
table.field_names = ["N \ m", "Эйлер 50", "Эйлер 100", "Эйлер 200", "Эйлер 500",
                    "Андерсен 50", "Андерсен 100", "Андерсен 200", "Андерсен 500", "Точная"]

analytical_price = heston_scalar(
    params['T'], params['K'], params['S0'], params['r'], True, heston_params
)


for N in N_values:
    row = [f"N={N}"]
    for m in m_values:
        price_euler = heston_euler_call(N=N, m=m, **params)
        row.append(f"{price_euler:.4f}")
    for m in m_values:
        price_qe = heston_qe_call(N=N, m=m, **params)
        row.append(f"{price_qe:.4f}")
    row.append(f"{analytical_price:.4f}")
    table.add_row(row)


print("Цены европейского опциона колл в модели Хестона (Эйлер vs Андерсен QE vs аналитика):")
print(table)

plt.figure(figsize=(10,6))
for m in m_values:
    prices_euler = []
    prices_qe = []
    N_plot = [1000, 2000, 5000, 10000, 20000]
    for N in N_plot:
        prices_euler.append(heston_euler_call(N=N, m=m, **params))
        prices_qe.append(heston_qe_call(N=N, m=m, **params))
    plt.plot(N_plot, prices_euler, 'o-', label=f'Эйлер m={m}')
    plt.plot(N_plot, prices_qe, '--o', label=f'Андерсен m={m}')
plt.axhline(analytical_price, color='r', linestyle='--', label='Интегральная оценка')
plt.xlabel('Число траекторий (N)')
plt.ylabel('Цена опциона')
plt.xscale('log')
plt.legend()
plt.grid()
plt.title('Сходимость Монте-Карло (Эйлер и QE-Андерсен) к точной цене')
plt.show()

_, S_paths, v_paths = heston_qe_call(
    N=10, m=100, return_paths=True, **params
)
t = np.linspace(0, params['T'], 101)
plt.figure(figsize=(12,8))


plt.subplot(2,1,1)
for i in range(5):
    plt.plot(t, S_paths[i])
plt.title('QE-Андерсен: траектории цен')
plt.xlabel('Время')
plt.ylabel('Цена')


plt.subplot(2,1,2)
for i in range(5):
    plt.plot(t, v_paths[i])
plt.title('QE-Андерсен: траектории волатильности')
plt.xlabel('Время')
plt.ylabel('Волатильность')
plt.tight_layout()
plt.show()

params = {
    'S0': 1,
    'v0': 0.09,
    'r': 0.05,
    'kappa': 2.0,
    'theta': 0.1,
    'sigma': 0.3,
    'rho': -0.5,
    'T': 1.0,
    'K': 1.1
}


heston_params = HestonParams(
    v=params['v0'],
    kappa=params['kappa'],
    theta=params['theta'],
    sigma=params['sigma'],
    rho=params['rho'],
)

N_values = [1000, 5000, 10000, 20000]
m_values = [50, 100, 200, 500]
table = PrettyTable()
table.field_names = ["N \ m", "Эйлер 50", "Эйлер 100", "Эйлер 200", "Эйлер 500",
                    "Андерсен 50", "Андерсен 100", "Андерсен 200", "Андерсен 500", "Точная"]

analytical_price = heston_scalar(
    params['T'], params['K'], params['S0'], params['r'], True, heston_params
)

for N in N_values:
    row = [f"N={N}"]
    for m in m_values:
        price_euler = heston_euler_call(N=N, m=m, **params)
        row.append(f"{price_euler:.4f}")
    for m in m_values:
        price_qe = heston_qe_call(N=N, m=m, **params)
        row.append(f"{price_qe:.4f}")
    row.append(f"{analytical_price:.4f}")
    table.add_row(row)

print("Цены европейского опциона колл в модели Хестона (Эйлер vs Андерсен QE vs аналитика):")
print(table)


plt.figure(figsize=(10,6))
for m in m_values:
    prices_euler = []
    prices_qe = []
    N_plot = [1000, 2000, 5000, 10000, 20000]
    for N in N_plot:
        prices_euler.append(heston_euler_call(N=N, m=m, **params))
        prices_qe.append(heston_qe_call(N=N, m=m, **params))
    plt.plot(N_plot, prices_euler, 'o-', label=f'Эйлер m={m}')
    plt.plot(N_plot, prices_qe, '--o', label=f'Андерсен m={m}')
plt.axhline(analytical_price, color='r', linestyle='--', label='Интегральная оценка')
plt.xlabel('Число траекторий (N)')
plt.ylabel('Цена опциона')
plt.xscale('log')
plt.legend()
plt.grid()
plt.title('Сходимость Монте-Карло (Эйлер и QE-Андерсен) к точной цене')
plt.show()

_, S_paths, v_paths = heston_qe_call(
    N=10, m=100, return_paths=True, **params
)
t = np.linspace(0, params['T'], 101)
plt.figure(figsize=(12,8))

plt.subplot(2,1,1)
for i in range(5):
    plt.plot(t, S_paths[i])
plt.title('QE-Андерсен: траектории цен')
plt.xlabel('Время')
plt.ylabel('Цена')

plt.subplot(2,1,2)
for i in range(5):
    plt.plot(t, v_paths[i])
plt.title('QE-Андерсен: траектории волатильности')
plt.xlabel('Время')
plt.ylabel('Волатильность')
plt.tight_layout()
plt.show()

params_asian = {
    'S0': 100,
    'v0': 0.04,
    'r': 0.05,
    'kappa': 2,
    'theta': 0.04,
    'sigma': 0.2,
    'rho': -0.7,
    'T': 1,
    'K': 100
}

def heston_asian_call_euler(S0, v0, r, kappa, theta, sigma, rho, T, K, N=10000, m=100):
    dt = T / m
    paths = np.zeros((N, m + 1))
    paths[:, 0] = S0
    v = np.full(N, v0)

    for t in range(1, m + 1):
        Z1 = np.random.normal(size=N)
        Z2 = np.random.normal(size=N)
        dW_S = np.sqrt(dt) * Z1
        dW_v = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)

        v = v + kappa * (theta - v) * dt + sigma * np.sqrt(np.maximum(v, 0)) * dW_v
        v = np.maximum(v, 0)

        paths[:, t] = paths[:, t-1] * np.exp((r - 0.5 * v) * dt + np.sqrt(v) * dW_S)

    avg_prices = np.mean(paths[:, 1:], axis=1)
    payoffs = np.maximum(avg_prices - K, 0)
    price = np.exp(-r * T) * np.mean(payoffs)
    return price

N_values_asian = [1000, 5000, 10000, 20000]
m_values_asian = [50, 100, 200]

table_asian = PrettyTable()
table_asian.field_names = ["N \\ m", "m=50", "m=100", "m=200"]

for N in N_values_asian:
    row = [f"N={N}"]
    for m in m_values_asian:
        price = heston_asian_call_euler(N=N, m=m, **params_asian)
        row.append(f"{price:.4f}")
    table_asian.add_row(row)

print("Цены азиатского опциона:")
print(table_asian)

calibrated_params_asian = {
    'S0': 100,
    'v0': 0.04,
    'r': 0.03,
    'kappa': 1.5,
    'theta': 0.05,
    'sigma': 0.25,
    'rho': -0.6,
    'T': 1,
    'K': 100
}


table_asian_calibrated = PrettyTable()
table_asian_calibrated.field_names = ["N \\ m", "m=50", "m=100", "m=200"]

for N in N_values_asian:
    row = [f"N={N}"]
    for m in m_values_asian:
        price = heston_asian_call_euler(N=N, m=m, **calibrated_params_asian)
        row.append(f"{price:.4f}")
    table_asian_calibrated.add_row(row)

print("\nЦены азиатского опциона (откалиброванные параметры):")
print(table_asian_calibrated)

# --- Код для форвард-стартинг опционов ---

params_forward = {
    'S0': 100,
    'v0': 0.04,
    'r': 0.05,
    'kappa': 2,
    'theta': 0.04,
    'sigma': 0.2,
    'rho': -0.7,
    'T1': 0.5, # Время фиксации страйка
    'T2': 1.0,  # Время экспирации
}

def heston_forward_start_call_euler(S0, v0, r, kappa, theta, sigma, rho, T1, T2, N=10000, m=100):
    dt = T2 / m
    m1 = int(m * T1 / T2)

    S = np.full(N, S0)
    v = np.full(N, v0)

    # Симуляция до T1
    for _ in range(m1):
        Z1 = np.random.normal(size=N)
        Z2 = np.random.normal(size=N)
        dW_S = np.sqrt(dt) * Z1
        dW_v = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)

        v_new = v + kappa * (theta - v) * dt + sigma * np.sqrt(np.maximum(v, 0)) * dW_v
        v = np.maximum(v_new, 0)

        S = S * np.exp((r - 0.5 * v) * dt + np.sqrt(v) * dW_S)

    # Фиксация страйка K = S(T1)
    K = S

    # Симуляция от T1 до T2
    for _ in range(m1, m):
        Z1 = np.random.normal(size=N)
        Z2 = np.random.normal(size=N)
        dW_S = np.sqrt(dt) * Z1
        dW_v = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)

        v_new = v + kappa * (theta - v) * dt + sigma * np.sqrt(np.maximum(v, 0)) * dW_v
        v = np.maximum(v_new, 0)

        S = S * np.exp((r - 0.5 * v) * dt + np.sqrt(v) * dW_S)

    payoffs = np.maximum(S - K, 0)
    price = np.exp(-r * T2) * np.mean(payoffs)
    return price

N_values_forward = [1000, 5000, 10000, 20000]
m_values_forward = [50, 100, 200]

table_forward = PrettyTable()
table_forward.field_names = ["N \\ m", "m=50", "m=100", "m=200"]

for N in N_values_forward:
    row = [f"N={N}"]
    for m in m_values_forward:
        price = heston_forward_start_call_euler(N=N, m=m, **params_forward)
        row.append(f"{price:.4f}")
    table_forward.add_row(row)

print("\nЦены форвард-стартинг опциона (оригинальные параметры):")
print(table_forward)

calibrated_params_forward = {
    'S0': 100,
    'v0': 0.04,
    'r': 0.03,
    'kappa': 1.5,
    'theta': 0.05,
    'sigma': 0.25,
    'rho': -0.6,
    'T1': 0.5,
    'T2': 1.0
}

table_forward_calibrated = PrettyTable()
table_forward_calibrated.field_names = ["N \\ m", "m=50", "m=100", "m=200"]

for N in N_values_forward:
    row = [f"N={N}"]
    for m in m_values_forward:
        price = heston_forward_start_call_euler(N=N, m=m, **calibrated_params_forward)
        row.append(f"{price:.4f}")
    table_forward_calibrated.add_row(row)

print("\nЦены форвард-стартинг опциона (откалиброванные параметры):")
print(table_forward_calibrated)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from dataclasses import dataclass
import scipy.stats as stats
import scipy.integrate as intg
import math
import cmath
import pandas as pd

def heston_euler_call(S0, v0, r, kappa, theta, sigma, rho, T, K, N=10000, m=100, return_paths=False):
    dt = T / m
    lnS = np.full((N, m+1), np.log(S0))
    v = np.full((N, m+1), v0)
    for t in range(1, m+1):
        Z1 = np.random.normal(size=N)
        Z2 = np.random.normal(size=N)
        dW1 = np.sqrt(dt) * Z1
        dW2 = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)
        v_prev = v[:, t-1]
        v_new = v_prev + kappa * (theta - v_prev) * dt + sigma * np.sqrt(np.abs(v_prev)) * dW2
        v[:, t] = np.maximum(v_new, 0)
        lnS[:, t] = lnS[:, t-1] + (r - 0.5 * v_prev) * dt + np.sqrt(np.abs(v_prev)) * dW1
    S_T = np.exp(lnS[:, -1])
    payoffs = np.maximum(S_T - K, 0)
    price = np.exp(-r * T) * np.mean(payoffs)
    if return_paths:
        S_paths = np.exp(lnS)
        return price, S_paths, v
    return price

def heston_qe_call(S0, v0, r, kappa, theta, sigma, rho, T, K, N=1000000, m=100, return_paths=False):
    dt = T / m
    lnS = np.full((N, m+1), np.log(S0))
    v = np.full((N, m+1), v0)
    for t in range(1, m+1):
        v_prev = v[:, t-1]
        m_v = theta + (v_prev - theta) * np.exp(-kappa * dt)
        s2_v = (v_prev * sigma ** 2 * np.exp(-kappa * dt) / kappa) * (1 - np.exp(-kappa * dt)) \
             + theta * sigma ** 2 * (1 - np.exp(-kappa * dt)) ** 2 / (2 * kappa)
        psi = s2_v / m_v ** 2
        Z = np.random.normal(size=N)
        U = np.random.uniform(size=N)
        v_new = np.zeros_like(v_prev)
        phi_c = 1.5
        mask = psi <= phi_c
        if np.any(mask):
            b2 = 2 / psi[mask] - 1 + np.sqrt(2 / psi[mask]) * np.sqrt(2 / psi[mask] - 1)
            a = m_v[mask] / (1 + b2)
            v_new[mask] = a * (np.sqrt(b2) + Z[mask]) ** 2
        if np.any(~mask):
            p = (psi[~mask] - 1) / (psi[~mask] + 1)
            beta = (1 - p) / m_v[~mask]
            u = U[~mask]
            v_new[~mask] = np.where(u <= p, 0, -np.log((1 - u) / (1 - p)) / beta)
        v[:, t] = v_new
        Z1 = np.random.normal(size=N)
        Z2 = np.random.normal(size=N)
        dW1 = np.sqrt(dt) * Z1
        dW2 = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho ** 2) * Z2)
        v_avg = (v_prev + v_new) / 2
        lnS[:, t] = lnS[:, t-1] + (r - 0.5 * v_avg) * dt + np.sqrt(np.abs(v_avg)) * dW1
    S_T = np.exp(lnS[:, -1])
    payoffs = np.maximum(S_T - K, 0)
    price = np.exp(-r * T) * np.mean(payoffs)
    if return_paths:
        S_paths = np.exp(lnS)
        return price, S_paths, v
    return price

@dataclass
class HestonParams:
    v: float
    kappa: float
    theta: float
    sigma: float
    rho: float

def heston_cf(s, v, kappa, theta, sigma, rho, u, t):
    d = cmath.sqrt((rho * sigma * u * 1j - kappa) ** 2 + sigma ** 2 * (u * 1j + u ** 2))
    g = ((rho * sigma * u * 1j - kappa + d) / (rho * sigma * u * 1j - kappa - d))
    C = (kappa * theta / sigma ** 2 * (
        t * (kappa - rho * sigma * u * 1j - d) -
        2 * cmath.log((1 - g * cmath.exp(-d * t)) / (1 - g))))
    D = ((kappa - rho * sigma * u * 1j - d) / sigma ** 2 *
         ((1 - cmath.exp(-d * t)) / (1 - g * cmath.exp(-d * t))))
    return cmath.exp(C + D * v + u * math.log(s) * 1j)

def heston_integrand(u, t, k, s, r, v, kappa, theta, sigma, rho):
    return (cmath.exp(-1j * u * math.log(k)) / (1j * u) *
            (cmath.exp(r * t) * heston_cf(s, v, kappa, theta, sigma, rho, u - 1j, t) -
             k * heston_cf(s, v, kappa, theta, sigma, rho, u, t))).real

def heston_scalar(t, k, s, r, is_call, heston_params):
    integral = intg.quad(
        heston_integrand, 0, 100,
        args=(t, k, s, r, heston_params.v, heston_params.kappa, heston_params.theta,
              heston_params.sigma, heston_params.rho), limit=200, epsabs=1e-8, epsrel=1e-8)[0]
    call_price = (0.5 * (s - math.exp(- r * t) * k) +
                  1 / math.pi * math.exp(- r * t) * integral)
    if is_call:
        return call_price
    return call_price + k * np.exp(- r * t) - s

def generate_and_save_prices(param_dict, filename, option_pricer_func, **kwargs):
    """Генерирует и сохраняет цены опционов в CSV."""
    S0 = param_dict['S0']
    r = param_dict['r']

    strikes = np.linspace(S0 * 0.8, S0 * 1.2, 50)
    maturities = np.linspace(0.1, 2.0, 50)

    results = []

    print(f"Начало расчета для файла: {filename}")

    for K in strikes:
        for T in maturities:
            if "european" in filename:
                heston_params_obj = HestonParams(v=param_dict['v0'], kappa=param_dict['kappa'], theta=param_dict['theta'], sigma=param_dict['sigma'], rho=param_dict['rho'])
                call_price = heston_scalar(T, K, S0, r, True, heston_params_obj)
                put_price = heston_scalar(T, K, S0, r, False, heston_params_obj)
            else:
                call_price = option_pricer_func(K=K, T=T, **param_dict, **kwargs)
                put_price = option_pricer_func(K=K, T=T, **param_dict, is_call=False, **kwargs)

            results.append({'strike': K, 'maturity': T, 'call': call_price, 'put': put_price})

    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)
    print(f"Файл {filename} успешно создан.")

#ЕВРОПЕЙСКИЕ ОПЦИОНЫ

params_european_orig = {
    'S0': 1, 'v0': 0.1, 'r': 0.045, 'kappa': 3.0, 'theta': 0.3, 'sigma': 0.5, 'rho': -0.2, 'T': 1.0, 'K': 0.8
}
generate_and_save_prices(params_european_orig, 'european_original_prices.csv', None)

# 2. Откалиброванные параметры
params_european_calib = {
    'S0': 1, 'v0': 0.09, 'r': 0.05, 'kappa': 2.0, 'theta': 0.1, 'sigma': 0.3, 'rho': -0.5, 'T': 1.0, 'K': 1.1
}
generate_and_save_prices(params_european_calib, 'european_calibrated_prices.csv', None)

#АЗИАТСКИЕ ОПЦИОНЫ

def heston_asian_option_euler(S0, v0, r, kappa, theta, sigma, rho, T, K, N=10000, m=100, is_call=True):
    dt = T / m
    paths = np.zeros((N, m + 1))
    paths[:, 0] = S0
    v = np.full(N, v0)

    for t in range(1, m + 1):
        Z1, Z2 = np.random.normal(size=N), np.random.normal(size=N)
        dW_S = np.sqrt(dt) * Z1
        dW_v = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)
        v = v + kappa * (theta - v) * dt + sigma * np.sqrt(np.maximum(v, 0)) * dW_v
        v = np.maximum(v, 0)
        paths[:, t] = paths[:, t-1] * np.exp((r - 0.5 * v) * dt + np.sqrt(v) * dW_S)

    avg_prices = np.mean(paths[:, 1:], axis=1)
    payoffs = np.maximum(avg_prices - K, 0) if is_call else np.maximum(K - avg_prices, 0)
    return np.exp(-r * T) * np.mean(payoffs)

#Оригинальные параметры
params_asian_orig = {
    'S0': 100, 'v0': 0.04, 'r': 0.05, 'kappa': 2, 'theta': 0.04, 'sigma': 0.2, 'rho': -0.7
}
generate_and_save_prices(params_asian_orig, 'asian_original_prices.csv', heston_asian_option_euler, N=1000, m=50)

#Откалиброванные параметры
params_asian_calib = {
    'S0': 100, 'v0': 0.04, 'r': 0.03, 'kappa': 1.5, 'theta': 0.05, 'sigma': 0.25, 'rho': -0.6
}
generate_and_save_prices(params_asian_calib, 'asian_calibrated_prices.csv', heston_asian_option_euler, N=1000, m=50)


#ФОРВАРД-СТАРТИНГ ОПЦИОНЫ

def heston_forward_start_option_euler(S0, v0, r, kappa, theta, sigma, rho, T, K, T1_ratio=0.5, N=10000, m=100, is_call=True):
    T1 = T * T1_ratio
    T2 = T
    dt = T2 / m
    m1 = int(m * T1 / T2)

    S = np.full(N, S0)
    v = np.full(N, v0)

    for _ in range(m1):
        Z1, Z2 = np.random.normal(size=N), np.random.normal(size=N)
        dW_S, dW_v = np.sqrt(dt) * Z1, np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)
        v = np.maximum(0, v + kappa * (theta - v) * dt + sigma * np.sqrt(np.maximum(v, 0)) * dW_v)
        S = S * np.exp((r - 0.5 * v) * dt + np.sqrt(v) * dW_S)

    Strike_forward = S

    for _ in range(m1, m):
        Z1, Z2 = np.random.normal(size=N), np.random.normal(size=N)
        dW_S, dW_v = np.sqrt(dt) * Z1, np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)
        v = np.maximum(0, v + kappa * (theta - v) * dt + sigma * np.sqrt(np.maximum(v, 0)) * dW_v)
        S = S * np.exp((r - 0.5 * v) * dt + np.sqrt(v) * dW_S)

    payoffs = np.maximum(S - Strike_forward, 0) if is_call else np.maximum(Strike_forward - S, 0)
    return np.exp(-r * T2) * np.mean(payoffs)

params_forward_orig = {
    'S0': 100, 'v0': 0.04, 'r': 0.05, 'kappa': 2, 'theta': 0.04, 'sigma': 0.2, 'rho': -0.7
}
generate_and_save_prices(params_forward_orig, 'forward_starting_original_prices.csv', heston_forward_start_option_euler, N=1000, m=50)

#Откалиброванные параметры
params_forward_calib = {
    'S0': 100, 'v0': 0.04, 'r': 0.03, 'kappa': 1.5, 'theta': 0.05, 'sigma': 0.25, 'rho': -0.6
}
generate_and_save_prices(params_forward_calib, 'forward_starting_calibrated_prices.csv', heston_forward_start_option_euler, N=1000, m=50)

print("\nВсе расчеты завершены и файлы сохранены.")